Environment setup.

In [ ]:
!pip install transformers
!pip install datasets

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [3]:
colab_data_path = "/content/drive/MyDrive/Seminar2/data/ribe_512x768/"
colab_dir = "/content/drive/MyDrive/Seminar2/model/"
model_name_or_path = "google/vit-base-patch16-224-in21k"

Imports.

In [4]:
import numpy as np
import torch
from torchvision import transforms
from datasets import load_dataset, concatenate_datasets
from transformers import (
    ViTForImageClassification,
    ViTFeatureExtractor,
    Trainer,
    TrainingArguments,
)
from transformers.trainer_utils import EvalPrediction
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score

Prepare dataset splits.

In [ ]:
dataset = load_dataset("imagefolder", data_dir=colab_data_path)
splits = dataset["train"].train_test_split(test_size=0.33)
dataset["train"] = splits["train"]
dataset["val"] = splits["test"]

Define image augmentations.

In [6]:
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4, hue=0.1),
    transforms.RandomRotation(30),
    transforms.RandomAffine(degrees=0, translate=(0.1, 0.1), scale=(0.8, 1.2)),
])

Apply the transforms to the train set. Concatenate the transformed and original train set.

In [7]:
transformed_train_dataset = dataset["train"].map(
    lambda example: {"image": transform(example["image"]), "label": example["label"]}
)

dataset["train"] = concatenate_datasets([transformed_train_dataset, dataset["train"]])

Map:   0%|          | 0/892 [00:00<?, ? examples/s]

Configure the Feature Extractor and apply it to the dataset.

In [ ]:
feature_extractor = ViTFeatureExtractor.from_pretrained(
    model_name_or_path,
    do_resize=False,
    patch_size=64,
)

def transform(example_batch):
    inputs = feature_extractor(
        [x.convert("RGB") for x in example_batch["image"]], return_tensors="pt"
      )
    inputs["labels"] = example_batch["label"]
    return inputs

dataset = dataset.with_transform(transform)

id2label = {id: label for id, label in enumerate(dataset["train"].features["label"].names)}
label2id = {label: id for id, label in id2label.items()}

Configure Vision Transformer model.

In [ ]:
# Set interpolate_pos_encoding=True in the source code.
model = ViTForImageClassification.from_pretrained(
    model_name_or_path,
    num_labels=len(id2label),
    id2label=id2label,
    label2id=label2id,
    ignore_mismatched_sizes=True,
)

model.config.hidden_dropout_prob = 0.5

Define auxiliary functions for the training procedure and configure training parameters.

In [10]:
def collate_fn(batch):
    return {
        'pixel_values': torch.stack([x['pixel_values'] for x in batch]),
        'labels': torch.tensor([x['labels'] for x in batch])
    }

def compute_metrics(eval_pred: EvalPrediction):
  preds = np.argmax(eval_pred.predictions, axis=1)
  return {
    "acc": accuracy_score(eval_pred.label_ids, preds),
    "f1": f1_score(eval_pred.label_ids, preds, average="weighted"),
    "precision": precision_score(eval_pred.label_ids, preds, average="weighted"),
    "recall": recall_score(eval_pred.label_ids, preds, average="weighted")
    }

In [11]:
training_args = TrainingArguments(
  output_dir=colab_dir,
  per_device_train_batch_size=8,
  per_device_eval_batch_size=8,
  evaluation_strategy="steps",
  num_train_epochs=6,
  fp16=True,
  save_steps=60,
  eval_steps=60,
  warmup_steps=500,
  logging_steps=60,
  learning_rate=2e-4,
  save_total_limit=2,
  remove_unused_columns=False,
  push_to_hub=False,
  report_to='tensorboard',
  load_best_model_at_end=True,
  weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=collate_fn,
    compute_metrics=compute_metrics,
    train_dataset=dataset["train"],
    eval_dataset=dataset["val"],
    tokenizer=feature_extractor,
)

Train the model.

In [12]:
train_results = trainer.train()
trainer.save_model()
trainer.log_metrics("train", train_results.metrics)
trainer.save_metrics("train", train_results.metrics)
trainer.save_state()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss,Validation Loss,Acc,F1,Precision,Recall
60,1.005800,0.654977,0.829545,0.788924,0.880356,0.829545
120,0.504900,0.268088,0.913636,0.907449,0.926164,0.913636
180,0.383100,0.229682,0.950000,0.950220,0.950704,0.950000
240,0.251500,0.257546,0.902273,0.905782,0.930443,0.902273
300,0.212300,0.363468,0.834091,0.834842,0.904093,0.834091
360,0.235100,0.269703,0.902273,0.895339,0.916548,0.902273
420,0.230400,0.353252,0.872727,0.877151,0.904670,0.872727
480,0.327300,0.195499,0.918182,0.919458,0.932433,0.918182
540,0.361700,0.102651,0.963636,0.963253,0.964019,0.963636
600,0.169000,0.123614,0.968182,0.967666,0.969186,0.968182


***** train metrics *****
  epoch                    =          6.0
  total_flos               = 6053995291GF
  train_loss               =       0.1919
  train_runtime            =   0:18:55.42
  train_samples_per_second =        9.427
  train_steps_per_second   =        1.178


Evaluate the model.

In [13]:
metrics = trainer.evaluate(dataset["test"])
trainer.log_metrics("eval", metrics)
trainer.save_metrics("eval", metrics)

***** eval metrics *****
  epoch                   =        6.0
  eval_acc                =     0.5286
  eval_f1                 =     0.5413
  eval_loss               =     2.4848
  eval_precision          =     0.6778
  eval_recall             =     0.5286
  eval_runtime            = 0:03:25.28
  eval_samples_per_second =      6.985
  eval_steps_per_second   =      0.877
